In [1]:
#importing the libraries which will be used later

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, time
from datetime import datetime as dt

In [2]:
#Getting the turnstile data from MTA

df1 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220507.txt")

df2 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220514.txt")

df3 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220521.txt")

df4 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220528.txt")

df5 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220604.txt")

In [3]:
#After importing weekly data, we have to convert into one table so we use pd.concat function

dfmay = pd.concat([df1,df2, df3, df4, df5], ignore_index=True) 

In [4]:
#Checking for null values as well as the Data types

dfmay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055980 entries, 0 to 1055979
Data columns (total 11 columns):
 #   Column                                                                Non-Null Count    Dtype 
---  ------                                                                --------------    ----- 
 0   C/A                                                                   1055980 non-null  object
 1   UNIT                                                                  1055980 non-null  object
 2   SCP                                                                   1055980 non-null  object
 3   STATION                                                               1055980 non-null  object
 4   LINENAME                                                              1055980 non-null  object
 5   DIVISION                                                              1055980 non-null  object
 6   DATE                                                                  1055980 non-

In [5]:
#Since we have 2 different columns as Date and Time, we are merging them into one datetime column 
#Using *datetime* library

datettimes = pd.to_datetime(dfmay["DATE"] + " " + dfmay["TIME"])

In [6]:
#inserting the datetime column into the dataframe

dfmay.insert(loc = 11,column = "datetime", value = datettimes )

In [7]:
#we have 3 different columns regarding the turnstiles so we merge them as well

turnstiles = dfmay["C/A"] + dfmay["UNIT"] + dfmay["SCP"]

In [8]:
#inserting the turnstiles column into our dataframe

dfmay.insert(loc = 12,column = "turnstile", value = turnstiles )

In [9]:
#Dropping irrelevant data from the datafram in order to enhance the efficiency

dfmay.drop(columns = ["C/A", "UNIT", "SCP", "DATE", "TIME", "DIVISION", "DESC", "LINENAME"], inplace = True)

In [10]:
#checking the dataframe

dfmay.columns = dfmay.columns.str.strip()

In [11]:
#again check for the datatypes since we have made some modifications regarding the dataframe

dfmay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055980 entries, 0 to 1055979
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   STATION    1055980 non-null  object        
 1   ENTRIES    1055980 non-null  int64         
 2   EXITS      1055980 non-null  int64         
 3   datetime   1055980 non-null  datetime64[ns]
 4   turnstile  1055980 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 40.3+ MB


In [12]:
#since the turnstile data increases cumulatively we have to take the difference between the rows by grouping them by stations

dfmay['net_entry']= dfmay.sort_values(['turnstile','datetime'],ascending = (False, True)).groupby(['turnstile'])['ENTRIES'].diff()
dfmay['net_exits']= dfmay.sort_values(['turnstile','datetime'],ascending = (False, True)).groupby(['turnstile'])['EXITS'].diff()

In [13]:
#these columns are not essential since already calculated the net exit and entry in [12]

dfmay.drop(columns = ["ENTRIES", "EXITS"], inplace = True)

In [14]:
#Another check for the net entries and exits for negative values

dfmay.describe()

,net_entry,net_exits
count,1.050933e+06,1.050933e+06
mean,-7.436687e+02,-2.042623e+03
std,4.221974e+05,2.043506e+06
min,-3.070229e+08,-2.061816e+09
25%,4.000000e+00,6.000000e+00
50%,2.800000e+01,3.800000e+01
75%,9.500000e+01,1.170000e+02
max,5.539190e+05,2.516514e+08


In [17]:
#Eliminating the negative values

dfmay = dfmay[dfmay['net_entry'] >= 0]
dfmay = dfmay[dfmay["net_exits"] >= 0]

In [18]:
#Checking the values again

dfmay.describe()

,net_entry,net_exits
count,1.038005e+06,1.038005e+06
mean,7.926246e+01,9.604153e+01
std,1.499026e+03,1.565741e+03
min,0.000000e+00,0.000000e+00
25%,4.000000e+00,7.000000e+00
50%,2.900000e+01,3.800000e+01
75%,9.600000e+01,1.170000e+02
max,2.621440e+05,9.150600e+05


In [19]:
#let's find the total number of people enters and exits the station

dfmay.insert(loc = 5, column = "total", value = dfmay["net_entry"] + dfmay["net_exits"])

In [20]:
#Whenever a column added we are checking for negative values in essence

dfmay.describe()

,net_entry,net_exits,total
count,1.038005e+06,1.038005e+06,1.038005e+06
mean,7.926246e+01,9.604153e+01,1.753040e+02
std,1.499026e+03,1.565741e+03,2.199799e+03
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,7.000000e+00,1.700000e+01
50%,2.900000e+01,3.800000e+01,9.000000e+01
75%,9.600000e+01,1.170000e+02,2.350000e+02
max,2.621440e+05,9.150600e+05,9.229610e+05


In [21]:
#Here is the final version of our dataframe

dfmay

,STATION,datetime,turnstile,net_entry,net_exits,total
1,59 ST,2022-04-30 04:00:00,A002R05102-00-00,4.0,4.0,8.0
2,59 ST,2022-04-30 08:00:00,A002R05102-00-00,3.0,27.0,30.0
3,59 ST,2022-04-30 12:00:00,A002R05102-00-00,21.0,89.0,110.0
4,59 ST,2022-04-30 16:00:00,A002R05102-00-00,46.0,39.0,85.0
5,59 ST,2022-04-30 20:00:00,A002R05102-00-00,125.0,43.0,168.0
...,...,...,...,...,...,...
1055975,RIT-ROOSEVELT,2022-06-03 05:00:00,TRAM2R46900-05-01,0.0,0.0,0.0
1055976,RIT-ROOSEVELT,2022-06-03 09:00:00,TRAM2R46900-05-01,0.0,0.0,0.0
1055977,RIT-ROOSEVELT,2022-06-03 13:00:00,TRAM2R46900-05-01,0.0,0.0,0.0
1055978,RIT-ROOSEVELT,2022-06-03 17:00:00,TRAM2R46900-05-01,0.0,2.0,2.0
